In [ ]:
# Importando as libraries

import requests
import pandas as pd
import json
from sqlalchemy import create_engine, Column, Integer, String, Float, Date, MetaData, Table, inspect
from sqlalchemy.orm import declarative_base, sessionmaker
from sqlalchemy.exc import SQLAlchemyError
import psycopg2

In [54]:
# Criando o dicionário com os endpoints e os descritivos simplificados

mapeamento_colunas = {
    "utilidades": {
        "1207": "pib_corrente",
        "21774": "populacao",
        "21775": "pib_per_capita",
    },
    "emprego": {
        "433":	"ipca",
        "1619": "salario_minimo",
        "24369": "taxa_desocupacao",
        "24370": "idade_trabalho",
        "24371": "empregados_total",
        "24372": "empregados_publico",
        "24373": "conta_propria",
        "24374": "empregadores",
        "24375": "empregados_carteira",
        "24376": "empregados_sem_carteira",
        "24377": "empregados_privado_total",
        "24378": "forca_trabalho",
        "24379": "pessoas_ocupadas",
        "24380": "pessoas_desocupadas",
        "28543": "ocupados_rendimento",
    }
}

In [55]:
# Definição das variáveis relevantes

# Datas de início e fim para a consulta
data_inicial = "01/01/2015"
data_final = ""

# URL base da API
BASE_URL = "https://api.bcb.gov.br/dados/serie/bcdata.sgs.{}/dados?formato=json&dataInicial={}&dataFinal={}"

In [ ]:
# Definição das funções relevantes

# Função para obter os dados da API e transformar em um pandas df
def obter_dados_api(chave):
    url = BASE_URL.format(chave, data_inicial, data_final)
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data)
        df["data"] = pd.to_datetime(df["data"], dayfirst=True)
        df["valor"] = pd.to_numeric(df["valor"], errors="coerce")
        df.rename(columns={"valor": str(chave)}, inplace=True)
        return df
    else:
        print(f"⚠️ Erro ao buscar dados para a chave {chave}")
        return None

# Função para processar um grupo de indicadores
def processar_grupo(grupo, colunas_mapeadas):
    endpoints = list(colunas_mapeadas.keys())
    dfs = [obter_dados_api(str(chave)) for chave in endpoints]
    dfs = [df for df in dfs if df is not None]  # Remove valores None

    if dfs:
        for df in dfs:
            df.set_index("data", inplace=True)

        df_concatenado = pd.concat(dfs, axis=1).reset_index()

        # Renomear colunas usando o mapeamento
        df_concatenado.rename(columns=colunas_mapeadas, inplace=True)

        # Remover linhas onde todas as colunas de indicadores são NaN
        df_concatenado.dropna(subset=colunas_mapeadas.values(), how="all", inplace=True)

        return df_concatenado
    return None

# Função para inserir DataFrame no PostgreSQL
def inserir_dataframe(df, tabela_nome):
    if df is None or df.empty:
        print(f"⚠️ DataFrame para {tabela_nome} está vazio. Nenhuma inserção feita.")
        return

    df.to_sql(tabela_nome, engine, if_exists='append', index=False)
    print(f"✅ Dados inseridos na tabela {tabela_nome}")

# Função principal
def main():
    resultados = {}

    for grupo, colunas_mapeadas in mapeamento_colunas.items():
        df_grupo = processar_grupo(grupo, colunas_mapeadas)
        if df_grupo is not None:
            resultados[grupo] = df_grupo
            print(f"✅ Dados do grupo '{grupo}' processados com sucesso!")

            # Inserir no PostgreSQL
            tabela_nome = f"indicadores_{grupo}"
            inserir_dataframe(df_grupo, tabela_nome)
        else:
            print(f"⚠️ Nenhum dado válido para o grupo '{grupo}'.")

    return resultados

In [ ]:
# Realizando a conexão do Python com o PostgreSQL usando o sqlalchemy

USERNAME = "postgres"
PASSWORD = "041700"
HOST = "localhost"  # Ou endereço do servidor
PORT = "5432"
DATABASE = "indicadores_teste"

# Criando a engine do SQLAlchemy
try:
    engine = create_engine(
        f"postgresql+psycopg2://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}",
        connect_args={"options": "-c client_encoding=UTF8"}
    )
    print("✅ Engine criada com sucesso!")
except Exception as e:
    print(f"❌ Criação da Engine falhou: {e}")

✅ Engine criada com sucesso!


In [59]:
# Executar a função principal
if __name__ == "__main__":
    dataframes_por_tema = main()

    # Exemplo: Acessando um DataFrame específico
    print("\n📊 Exemplo de DataFrame para 'emprego':")
    print(dataframes_por_tema.get("emprego", "Nenhum dado encontrado."))

✅ Dados do grupo 'utilidades' processados com sucesso!
✅ Dados inseridos na tabela indicadores_utilidades
✅ Dados do grupo 'emprego' processados com sucesso!
✅ Dados inseridos na tabela indicadores_emprego

📊 Exemplo de DataFrame para 'emprego':
          data  ipca  salario_minimo  taxa_desocupacao  idade_trabalho  \
0   2015-01-01  1.24           788.0               6.9        159848.0   
1   2015-02-01  1.22           788.0               7.5        160018.0   
2   2015-03-01  1.32           788.0               8.0        160189.0   
3   2015-04-01  0.71           788.0               8.1        160354.0   
4   2015-05-01  0.74           788.0               8.3        160525.0   
..         ...   ...             ...               ...             ...   
118 2024-11-01  0.39          1412.0               6.1        176689.0   
119 2024-12-01  0.52          1412.0               6.2        176810.0   
120 2025-01-01  0.16          1518.0               6.5        176930.0   
121 2025-02-01